In [ ]:
import torch
from torch import nn
# define customized functions with customized gradients
class STEFunction_relay(torch.autograd.Function):
    """ define straight through estimator with overrided gradient (gate) """
    @staticmethod
    def forward(ctx, input_x, mask, threshold = 0.005):
        ctx.save_for_backward(input_x)
        mask_higher = (input_x > threshold).float()
        mask_lower = (input_x < (-1) * threshold).float()
        mask > 0
        return (input_x > 0).float()

    @staticmethod
    def backward(ctx, grad_output):
        input_x, = ctx.saved_tensors
        return torch.mul(F.softplus(input_x), grad_output)

In [ ]:
now_state = 1
in_val = -0.015
threshold = 0.01
def Hysteresis(now_state, in_val, threshold):
    if now_state == 1:
        if in_val < (-1) * threshold:
            now_state = 0
    else:
        if in_val > threshold:
            now_state = 1
    return now_state
now_state = Hysteresis(now_state, in_val, threshold) 
print(now_state)

In [ ]:
now_state = 1.0
in_val_list = [-0.011, 0.005, 0.001, 0.011, -0.009, -0.011, 0.0, 0.011]
threshold = 0.01
def Hysteresis(now_state, in_val, threshold):
    now_state = now_state * float(in_val > (-1) * threshold) + (1 - now_state) * float(in_val > threshold)
    return now_state
for in_val in in_val_list:
    now_state = Hysteresis(now_state, in_val, threshold) 
    print(now_state)

In [ ]:
now_state = 0
in_val = -0.0101
threshold = 0.01
now_state = now_state * float(in_val > (-1) * threshold) + (1 - now_state) * float(in_val > threshold)
print(now_state)

In [ ]:
import torch
from torch.nn import functional as F
class MultiInputRelayFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input1, input2):
        ctx.save_for_backward(input1, input2)
        return input1, input2

    @staticmethod
    def backward(ctx, grad_output1, grad_output2):
        input1, input2 = ctx.saved_tensors
        if not input1.requires_grad:
            grad_input1 = None
        else:
            grad_input1 = grad_output1.clone()

        if not input2.requires_grad:
            grad_input2 = None
        else:
            grad_input2 = grad_output2.clone()

        return grad_input1, grad_input2

x1 = torch.randn(10, requires_grad=True)
x2 = torch.randn(10)
print(x1)
# Use the MultiInputRelayFunction to calculate the output
output = MultiInputRelayFunction.apply(x1, x2)

# Calculate the loss
loss = (output[0] ** 2).sum()

# Backprop to calculate gradients
loss.backward()

# Update x1 using an optimizer
optimizer = torch.optim.Adam([x1], lr=0.1)
optimizer.step()
print(x1)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# define customized functions with customized gradients
class STEFunction_relay(torch.autograd.Function):
    """ define straight through estimator with overrided gradient (gate) """
    @staticmethod
    def forward(ctx, input, mask_old):
        ctx.save_for_backward(input)
        return mask_old

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        return torch.mul(F.softplus(input), grad_output), None

x1 = torch.randn(10, requires_grad=True)
nn.init.uniform_(x1, a = 0, b = 1)
mask_old = (x1 > 0).float()
print(x1)
print(mask_old)
mask_old.requires_grad = False
x1_out = STEFunction_relay.apply(x1, mask_old)
loss = torch.sum(x1_out)
threshold = 0.5

loss.backward()
optimizer = torch.optim.Adam([x1], lr=1)
optimizer.step()
print(x1)
print(mask_old)
mask_new = mask_old * (x1 > (-1) * threshold).float() + (1 - mask_old) * (x1 > threshold).float()
print(mask_new)

In [ ]:
import torch

# Create a tensor with random values
tensor = torch.rand(3, 3)

# Define a model with a single linear layer
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(3, 3)

# Initialize the model
model = Model()

# Copy the values from the tensor into the model's parameters
print(model.linear.weight)
print(tensor)
model.linear.weight.data.copy_(tensor)
print(model.linear.weight)

### Other method to copy parameter from one to another
param1 = torch.nn.Parameter(torch.rand(3, 3))
param2 = torch.nn.Parameter(torch.rand(3, 3))
param2.data.copy_(param1.data)
print(param2.data)


In [ ]:
class MyClass:
    def __init__(self):
        self._attr1 = Attr1()
    def set_attr1(self, value):
        self._attr1.attr2 = value

class Attr1:
    def __init__(self):
        self.attr2 = None

obj = MyClass()
obj.set_attr1("Value")
print(obj._attr1.attr2)
# setattr(eval('obj._attr1'), 'attr2', 'a')
print(getattr(obj, '_attr1').attr2)
getattr(obj, '_attr1').attr2 = 'a'
print(obj._attr1.attr2)
obj._attr1.attr2 = 'a'
print(obj._attr1.attr2)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.w1 = nn.Parameter(torch.randn(1))
        self.w2 = nn.Parameter(torch.randn(1))
        self.b = nn.Parameter(torch.randn(1))

    def forward(self, x):
        fx = nn.functional.relu(x)
        # gx = self.w2 * (x ** 2) + self.w1 * x + self.b
        gx = self.w1 * x + self.b
        return fx, gx

# Define your loss function
def loss_fn(fx, gx):
    return (fx - gx).pow(2).mean()

net = Net()
# Define your optimizer
optimizer = optim.Adam([net.w1, net.w2, net.b], lr=0.001)
epochs = 10000
lr_scheduler_w = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer, T_max = epochs)
# Train the model
for i in range(10000):
    # Generate a new random x with normal distribution
    x = torch.randn(32, 32) #N(0, 1)
    # x += torch.randn(32, 32) #N(0, 2)
    fx, gx = net(x)
    loss = loss_fn(fx, gx)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    lr_scheduler_w.step()
    print(loss.item())

In [ ]:
print(net.w2, net.w1, net.b)

In [ ]:
# x = torch.randn(20)
x = torch.tensor([3.0])
# x += torch.randn(20)
fx, gx = net(x)
print(x)
print(fx)
print(gx)

In [ ]:
x = torch.tensor([3.0])
x.var_map = 1
print(x.var_map)
fx, gx = net(x)
print(fx.var_map)

In [ ]:
import torch
from torch.nn import functional as F
x = torch.randn(3, 3)
print(x)
mask = (torch.randn(3, 3) > 0).float()
print(mask)
out1 = F.relu(x * mask)
out2 =  mask* F.relu(x)
print(out1)
print(out2)
print(out1 - out2)


In [ ]:
import torch
from torch.nn import functional as F
import torch.optim as optim

with torch.no_grad():
    x0 = torch.randn(4, 4)
    x1 = torch.clone(x0)
    mask0 = (torch.randn(4, 4) > 0).float()
    mask1 = torch.clone(mask0)
x0.requires_grad = True
x1.requires_grad = True
mask0.requires_grad = True
mask1.requires_grad = True
optimizer0 = optim.Adam([x0, mask0], lr=1)
optimizer1 = optim.Adam([x1, mask1], lr=1)
for i in range(1):
    out0 = F.relu(x0 * mask0)
    out1 =  mask1* F.relu(x1)
    loss0 = torch.sum(out0)
    loss1 = torch.sum(out1)
    print("difference between out0 and out1:", torch.sum(out0 - out1))
    optimizer0.zero_grad()
    optimizer1.zero_grad()
    loss0.backward()
    loss1.backward()
    print(mask0.grad)
    print(mask1.grad)
    optimizer0.step()
    optimizer1.step()
    # print(x0)
    # print(x1)
    # print(mask0)
    # print(mask1)

In [ ]:
import torch
x = torch.tensor([1, 2, 3])
x.size()
x.expand(4, 3)